In [55]:
# Imports
import json, re, numpy as np, numpy.linalg as nplin, matplotlib.pyplot as plt, matplotlib.mlab as mlab, scipy.stats as spstat
from __future__ import division
from pyspark.mllib.feature import HashingTF, IDF, Normalizer
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, RandomForest, GradientBoostedTrees
from operator import itemgetter
from time import time
%matplotlib inline

In [42]:
# Load data, split into test and training set
all_reviews = sc.textFile("s3n://stat-37601/ratings.json", minPartitions=1000).map(json.loads)
reviews, reviews_test = all_reviews.randomSplit([.1, .9])
reviews.cache()

PythonRDD[91] at RDD at PythonRDD.scala:42

Get the variable we are regressing on: a continious score out of 1

In [43]:
# Get the variable we are regressing on, the review as a score out of 1
def getLabel(review):
    """Get the overall rating from a review"""
    label, total = review["review_overall"].split("/")
    return float(label) / float(total)

labels      = reviews.map(getLabel)
labels_test = reviews_test.map(getLabel)

#### (a) Generating features (hashed TF-IDF)
We generate a list of words (the features) for each review, transform them into hashes, calculate term frequency-inverse document frequency accross corpus, and normalize

In [44]:
# Parser, mostly from earlier problem using SGD on tweets, except without code for emoticions

# words to ignore
stop = set(['the', 'and', 'you', 'your', 'for', 'por', 'que', 'las', 'los', 'les',\
       'una', 'del', 'este', 'usted', 'para', 'con', 'this', 'that', 'was', 'have', 'like',\
       'would', 'could', 'should', 'will', 'can', 'shall', 'just', 'all', 'it', 'its', 'per'])
eng_stop = set(['i', 'me', 'my', 'myself', 'we', 'our', \
             'ours', 'ourselves', 'you', 'your', 'yours', \
             'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', \
             'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', \
             'themselves', 'what', 'which', 'who', 'whom', 'this', \
             'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', \
             'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do',\
             'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', \
             'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at',\
             'by', 'for', 'with', 'about', 'against', 'between', 'into', \
             'through', 'during', 'before', 'after', \
            'above', 'below', 'to', 'from', 'up', 'down', 'in',\
            'out', 'on', 'off', 'over', 'under', 'again', 'further', \
            'then', 'once', 'here', 'there', 'when', 'where', 'why', \
            'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',\
            'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', \
            'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', \
            'just', 'dont', 'should', 'now','on'])
spa_stop = set()
all_stop = stop|eng_stop|spa_stop

# word processor function
def splitter(s,ignore=all_stop):
    s = re.sub("([a-zA-Z])'([a-zA-Z])","\g<1>\g<2>",s) # standardize to no apostrophe
    s = re.sub('[^a-zA-Z!\?]',' ',s)           # get rid of most punctuation 
    s = re.sub('\?![\?!]*|!\?[\?!]*',' !? ',s) # standardize ?!?!?!
    s = re.sub('!+','!',s)                    # standardize to single !
    s = re.sub('\?+','?',s)                   # standarize to single ?
    s = re.sub('([a-zA-z]{2,})([?!]+)(\s|$)','\g<1> \g<2> ',s) # single out punctuation
    s = re.sub('(?!http://)www\.\S+|http://\S+','',s) # get rid of urls
    return list([w.lower() for w in s.split() if w not in ignore])

In [45]:
# Do the hashing transform
revHTF = HashingTF(numFeatures=1000)
reviewFrequency      = revHTF.transform(     reviews.map(lambda review: splitter(review["review_text"]))).cache()
review_testFrequency = revHTF.transform(reviews_test.map(lambda review: splitter(review["review_text"]))).cache()

# Do the inverse document frequency transform
revIDF = IDF().fit(reviewFrequency)
nor = Normalizer()
features      = nor.transform(revIDF.transform(reviewFrequency)).cache()
features_test = nor.transform(revIDF.transform(review_testFrequency)).cache()

# Un-cache unneeded data sets
reviewFrequency.unpersist()
review_testFrequency.unpersist()

PythonRDD[93] at RDD at PythonRDD.scala:42

In [46]:
# Join the labels back with the features
data = features.zip(labels).map(lambda (feature, label): LabeledPoint(label, feature))

#### (b) The regression model and evaluation
Write a function to compute MSE for training and test datasets

In [53]:
def treeMSE(tree,train_feat=features,train_label=labels,test_feat=features_test,test_label=labels_test):
    '''Evaluates training and test error for a pyspark mllib tree model'''
    train_MSE = train_label.zip(tree.predict(train_feat)).map(lambda (l,p):(l-p)**2).sum() / train_label.count()
    test_MSE  = test_label.zip(tree.predict(test_feat)).map(lambda (l,p):(l-p)**2).sum() / test_label.count()
    return(train_MSE,test_MSE)

####(c) train and test some trees
Test out some parameterizations of gradient boosted trees and random forrests

In [60]:
model = RandomForest.trainRegressor(data=data,categoricalFeaturesInfo={},numTrees=3,impurity='variance',maxDepth=2)

KeyboardInterrupt: 